# [Generate synthetic and simulated data for evaluation](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/simulator-interaction-data)
**Azure AI Evaluation SDK's** `Simulator` provides an end-to-end synthetic data generation capability to help developers test their application's response to typical user queries in the absence of production data. AI developers can use an index or text-based query generator and fully customizable simulator to create robust test datasets around non-adversarial tasks specific to their application. The `Simulator` class is a powerful tool designed to generate synthetic conversations and simulate task-based interactions. This capability is useful for:
- **Testing Conversational Applications**: Ensure your chatbots and virtual assistants respond accurately under various scenarios.
- **Training AI Models**: Generate diverse datasets to train and fine-tune machine learning models.
- **Generating Datasets**: Create extensive conversation logs for analysis and development purposes.
By automating the creation of synthetic data, the Simulator class helps streamline the development and testing processes, ensuring your applications are robust and reliable.
<br/>
By automating the creation of synthetic data, the `Simulator` class helps streamline the development and testing processes, ensuring your applications are robust and reliable.

In [1]:
# !az login

In [2]:
# Constants and Libraries
import os, json
from azure.identity import DefaultAzureCredential, get_bearer_token_provider #requires azure-identity
from pprint import pprint
from dotenv import load_dotenv # requires python-dotenv
from typing import List, Dict, Any, Optional
from promptflow.client import load_flow
from pprint import pprint

if not load_dotenv("./../../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
    sys.exit()
os.environ["AZURE_OPENAI_API_VERSION"] = os.environ["OPENAI_API_VERSION"]

credential = DefaultAzureCredential()

In [3]:
# Initialize Azure OpenAI connection

model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("MODEL_DEPLOYMENT_NAME"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
    "type": "AzureOpenAI" # NEEDED FOR \Lib\site-packages\promptflow\core\_prompty_utils.py
}

## Specify application Prompty
The following `application.prompty` file specifies how a chat application behaves.

In [4]:
%%writefile ./eval_assets/application.prompty
---
name: ApplicationPrompty
description: Chat RAG application
model:
    api: chat
    parameters:
        temperature: 0.0
        top_p: 1.0
        presence_penalty: 0
        frequency_penalty: 0
        response_format:
            type: text
 
inputs:
    context:
        type: string
    query:
        type: string
    conversation_history:
        type: dict
---
system:
You are a helpful assistant and you're helping with the user's query. 
Keep the conversation engaging and interesting.

Keep your conversation grounded in the provided context: 
{{ context }}

Output with a string that continues the conversation, responding to 
the latest message from the user query using it same language:
{{ query }}

given the conversation history:
{{ conversation_history }}

Overwriting ./eval_assets/application.prompty


In [5]:
prompty_path = "eval_assets/application.prompty"
flow = load_flow(source=prompty_path, model={"configuration": model_config})

pprint(flow(context="", query="How to make a pizza at home", conversation_history=[]))

("Making a pizza at home is a fun and delicious activity! Here's a simple way "
 'to do it:\n'
 '\n'
 '1. **Prepare the Dough**: You can either buy pre-made dough or make your '
 'own. For homemade dough, mix 2 cups of flour, 1 packet of yeast, 1 teaspoon '
 'of salt, 1 tablespoon of olive oil, and 3/4 cup of warm water. Knead until '
 'smooth and let it rise for about an hour.\n'
 '\n'
 '2. **Roll Out the Dough**: Once the dough has risen, roll it out on a '
 'floured surface to your desired thickness.\n'
 '\n'
 '3. **Add Sauce**: Spread a layer of tomato sauce over the dough. You can use '
 'store-bought or make your own by simmering tomatoes, garlic, and herbs.\n'
 '\n'
 '4. **Top It Off**: Add your favorite toppings! Classic choices include '
 'mozzarella cheese, pepperoni, mushrooms, bell peppers, and onions.\n'
 '\n'
 '5. **Bake**: Preheat your oven to 475°F (245°C). Bake the pizza for about '
 '10-15 minutes, or until the crust is golden and the cheese is bubbly.\n'
 '\n'
 '6. *

In [6]:
# English version

context = "Friendly conversation between two people, in English"

conversation_history = [
    {
        "content": "What are you up to this evening?",
        "role": "user",
        "context": "Friendly question",
    },
    {
        "content": "I'm going to the cinema",
        "role": "assistant",
        "context": "Description of a fun recreational activity",
    },
    {
        "content": "Ah fantastic! What are you going to see?",
        "role": "user",
        "context": "Follow-up friendly question showing interest in the other person",
    },
    {
        "content": "JAWS by Steven Spielberg",
        "role": "assistant",
        "context": "Specific information about a fun activity one plans to do",
    },
]

# flow(context=conversation_history[-1]["context"], query=conversation_history[-1]["content"], conversation_history=conversation_history)

In [12]:
i = 0
for ch in conversation_history:
    print(f'Message {i} - {ch["role"]}: {ch["content"]}\n')
    i += 1
    
response = flow(context=conversation_history[-1]["context"], query=conversation_history[-1]["content"], conversation_history=conversation_history)

conversation_history.append({"content": response, "role": "user" if conversation_history[-1]["role"]=="assistant" else "assistant", "context": ""})
print(f'Message {i} - {conversation_history[-1]["role"]}: {conversation_history[-1]["content"]}\n')

Message 0 - user: What are you up to this evening?

Message 1 - assistant: I'm going to the cinema

Message 2 - user: Ah fantastic! What are you going to see?

Message 3 - assistant: JAWS by Steven Spielberg

Message 4 - user: Oh, that's a classic! JAWS is such a thrilling movie. Are you a fan of Spielberg's work, or is this your first time watching it?

Message 5 - assistant: I've seen a few of Spielberg's films, but this is my first time watching JAWS on the big screen. I'm really excited to experience it in a theater setting! How about you? Do you have a favorite Spielberg movie?

Message 6 - user: Spielberg has so many great films, it's hard to pick just one! I think "Jurassic Park" might be my favorite. The mix of adventure, suspense, and groundbreaking effects really captivated me. How about you? Do you have a Spielberg favorite that stands out?

Message 7 - assistant: "Jurassic Park" is definitely a masterpiece! For me, "E.T. the Extra-Terrestrial" holds a special place in my he

## Specify target callback to simulate against
You can bring any application endpoint to simulate against by specifying a target callback function such as the following given an application that is an LLM with a Prompty file like `application.prompty`

In [13]:
async def callback(
    messages: Dict,
    stream: bool = False,
    session_state: Any = None,  # noqa: ANN401
    context: Optional[Dict[str, Any]] = None,
    subfolder: str = "eval_assets",
) -> dict:
    messages_list = messages["messages"]
    latest_message = messages_list[-1] # Get the last message
    query = latest_message["content"]
    context = latest_message.get("context", None) # looks for context, default None
    # Call your endpoint or AI application here
    prompty_path = os.path.join(os.getcwd(), subfolder, "application.prompty") 
    _flow = load_flow(source=prompty_path, model={"configuration": model_config})
    response = _flow(query=query, context=context, conversation_history=messages_list)
    # Format the response to follow the OpenAI chat protocol
    formatted_response = {
        "content": response,
        "role": "user" if conversation_history[-1]["role"]=="assistant" else "assistant",
        "context": context,
    }
    messages["messages"].append(formatted_response)
    return {
        "messages": messages["messages"],
        "stream": stream,
        "session_state": session_state,
        "context": context
    }

## Generate text or index-based synthetic data as input
In the first part, we prepare the text for generating the input to our simulator

In [14]:
import asyncio
from azure.identity import DefaultAzureCredential
import wikipedia
import os
# Prepare the text to send to the simulator
wiki_search_term = "Leonardo da Vinci"
wiki_title = wikipedia.search(wiki_search_term)[0]
wiki_page = wikipedia.page(wiki_title)
wiki_text = wiki_page.summary[:5000]
print(f"{wiki_text[:100]}...")

Leonardo di ser Piero da Vinci (15 April 1452 - 2 May 1519) was an Italian polymath of the High Rena...


In [15]:
from azure.ai.evaluation.simulator import Simulator
simulator = Simulator(model_config=model_config)

Class Simulator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [16]:
outputs = await simulator(
    num_queries = 1,  # Number of queries
    text        = wiki_text,
    target      = callback   
)

D:\ProgramData\miniconda3\envs\genai_evaluation\Lib\site-packages\azure\ai\evaluation\simulator\_simulator.py:154: UserWarning: You have specified 'num_queries' > len('tasks') (1 > 0). All tasks will be used for generation and the remaining 1 lines will be simulated in task-free mode
  warnings.warn(
Generating: 100%|████████████████████████████████████████████████| 5/5 [00:26<00:00,  5.32s/message]


In [17]:
outputs

[{'messages': [{'role': 'user',
    'content': 'When was Leonardo da Vinci born?',
    'context': 'None'},
   {'role': 'assistant',
    'content': "Leonardo da Vinci was born on April 15, 1452. He was a true Renaissance man, known for his incredible contributions to art, science, and engineering. Is there anything specific about his life or work that you're curious about?",
    'context': 'None'},
   {'role': 'user',
    'content': "I'm curious about his contributions to engineering. Could you tell me more about his inventions or designs?",
    'context': 'None'},
   {'role': 'assistant',
    'content': "Leonardo da Vinci was a visionary in engineering, and his notebooks are filled with sketches and ideas that were far ahead of his time. Some of his most fascinating designs include the flying machine, which resembles a modern-day helicopter, and the armored vehicle, an early concept of a tank. He also designed various machines for water and hydraulic systems, such as a water wheel and 

In [ ]:
i = 0
for ch in outputs[0]["messages"]:
    print(f'Message {i} - {ch["role"]}: {ch["content"]}\n')
    i += 1